## Tratamento dos dados
---
Inicialmente, temos os dados "crus", retirados do IBGE. Estes dados mostram o valor monetário da produção de cada um dos setores, e em que parte da cadeia produtiva ele se encontra.

In [1]:
import re

import pandas as pd
import numpy as np

In [2]:
# link da tabela:
# https://www.ibge.gov.br/estatisticas/economicas/contas-nacionais/9085-matriz-de-insumo-produto.html?=&t=resultados
matriz = pd.read_excel("Matriz_de_Insumo_Produto_2015_Nivel_67.xls",
                       "03", skiprows=3)
matriz

Unnamed: 0  \
0                                                  NaN   
1                                                01911   
2                                                01912   
3                                                01913   
4                                                01914   
..                                                 ...   
127                                              97001   
128                                                NaN   
129                                              Total   
130                                                NaN   
131  Fonte: IBGE, Diretoria de Pesquisas, Coordenaç...   

                                            Unnamed: 1  Valor da produção  \
0                                                  NaN                NaN   
1                        Arroz, trigo e outros cereais            11036.0   
2                                        Milho em grão            29975.0   
3    Algodão herbáceo, outras fibras da lavoura tem...             8943.0   
4                                       Cana-de-açúcar            46080.0   
..                                                 ...                ...   
127                                Serviços domésticos            61996.0   
128                                                NaN                NaN   
129                                                NaN         10226869.0   
130                                                NaN                NaN   
131                                                NaN                NaN   

     0191\nAgricultura, inclusive o apoio à agricultura e a pós-colheita  \
0                                                  NaN                     
1                                                166.0                     
2                                                566.0                     
3                                                260.0                     
4                                                615.0                     
..                                                 ...                     
127                                                0.0                     
128                                                NaN                     
129                                           115679.0                     
130                                                NaN                     
131                                                NaN                     

     0192\nPecuária, inclusive o apoio à pecuária  \
0                                             NaN   
1                                           164.0   
2                                          2314.0   
3                                            23.0   
4                                           148.0   
..                                            ...   
127                                           0.0   
128                                           NaN   
129                                       55512.0   
130                                           NaN   
131                                           NaN   

     0280\nProdução florestal; pesca e aquicultura  \
0                                              NaN   
1                                              0.0   
2                                              9.0   
3                                              0.0   
4                                              4.0   
..                                             ...   
127                                            0.0   
128                                            NaN   
129                                         6546.0   
130                                            NaN   
131                                            NaN   

     0580\nExtração de carvão mineral e de minerais não metálicos  \
0                                                  NaN              
1                                                  0.0              
2               

### Problemas a serem resolvidos: 

- Colunas com texto e valores ``NaN``;

- A maior parte das colunas tem números e um '\n' no nome;

- Os setores estão subdivididos (não é uma matriz quadrada);

Primeiro, apagamos as linhas desnecessárias

In [3]:
matriz.drop(index=[0, 128, 130, 131], inplace=True)
matriz

Unnamed: 0                                         Unnamed: 1  \
1        01911                      Arroz, trigo e outros cereais   
2        01912                                      Milho em grão   
3        01913  Algodão herbáceo, outras fibras da lavoura tem...   
4        01914                                     Cana-de-açúcar   
5        01915                                      Soja  em grão   
..         ...                                                ...   
124      94801  Organizações patronais, sindicais e outros ser...   
125      94802  Manutenção de computadores, telefones e objeto...   
126      94803                                  Serviços pessoais   
127      97001                                Serviços domésticos   
129      Total                                                NaN   

     Valor da produção  \
1              11036.0   
2              29975.0   
3               8943.0   
4              46080.0   
5             109170.0   
..                 ...   
124            83334.0   
125            26429.0   
126            42366.0   
127            61996.0   
129         10226869.0   

     0191\nAgricultura, inclusive o apoio à agricultura e a pós-colheita  \
1                                                166.0                     
2                                                566.0                     
3                                                260.0                     
4                                                615.0                     
5                                               1864.0                     
..                                                 ...                     
124                                              104.0                     
125                                                0.0                     
126                                                0.0                     
127                                                0.0                     
129                                           115679.0                     

     0192\nPecuária, inclusive o apoio à pecuária  \
1                                           164.0   
2                                          2314.0   
3                                            23.0   
4                                           148.0   
5                                           155.0   
..                                            ...   
124                                          29.0   
125                                           0.0   
126                                           0.0   
127                                           0.0   
129                                       55512.0   

     0280\nProdução florestal; pesca e aquicultura  \
1                                              0.0   
2                                              9.0   
3                                              0.0   
4                                              4.0   
5                                              0.0   
..                                             ...   
124                                           14.0   
125                                            5.0   
126                                            0.0   
127                                            0.0   
129                                         6546.0   

     0580\nExtração de carvão mineral e de minerais não metálicos  \
1                                                  0.0              
2                                                  0.0              
3                                                  0.0              
4                                                  0.0              
5                                                  0.0              
..                                                 ...              
124                                               35.0              
125                                                0.0              
126                                       

In [4]:
colunas = list(matriz.columns)

matriz_insumo_produto = pd.DataFrame(columns=matriz.columns)

for i in range(len(colunas)):
    """
    Se verificar, irá ver que "0191 Agricultura, inclusive o apoio à agricultura e o pós-colheita"
    está dividido em 10 partes, todas com um código começando com 0191!
    Isso permite a automatização da soma das subdivisões setoriais.
    """
    indices = []
    
    for j in range(1, len(matriz.index)):
        
        subsetor = matriz["Unnamed: 0"][j]
        
        if subsetor[:4] in list(matriz.columns)[i]:
            # se o começo do código bate com o código desta coluna, marque-a para ser adicionada
            indices.append(matriz.index[j-1])
            
    if len(indices) != 0:
        # se há colunas para serem somadas, as some e as adicione na matriz_insumo_produto
        matriz_insumo_produto = matriz_insumo_produto.append(
            matriz.loc[indices[0]:indices[-1]].sum(),
            ignore_index=True
        )
        
matriz_insumo_produto

Unnamed: 0  \
0   019110191201913019140191501916019170191801919   
1                            01921019220192301924   
2                                      0280102802   
3                                      0580105802   
4                                           06801   
..                                            ...   
62                                          86911   
63                                          86921   
64                                          90801   
65                                948019480294803   
66                                          97001   

                                           Unnamed: 1  Valor da produção  \
0   Arroz, trigo e outros cereaisMilho em grãoAlgo...           302321.0   
1   Bovinos e outros animais vivos, produtos anima...           128920.0   
2   Produtos da exploração florestal e da silvicul...            34101.0   
3                Carvão mineralMinerais não metálicos            20365.0   
4           Petróleo, gás natural e serviços de apoio           162130.0   
..                                                ...                ...   
62                                      Saúde pública           175278.0   
63                                      Saúde privada           224955.0   
64    Serviços de artes, cultura, esporte e recreação            33796.0   
65  Organizações patronais, sindicais e outros ser...           152129.0   
66                                Serviços domésticos            61996.0   

    0191\nAgricultura, inclusive o apoio à agricultura e a pós-colheita  \
0                                              6657.0                     
1                                               521.0                     
2                                               915.0                     
3                                                46.0                     
4                                                 0.0                     
..                                                ...                     
62                                                0.0                     
63                                                0.0                     
64                                                0.0                     
65                                              104.0                     
66                                                0.0                     

    0192\nPecuária, inclusive o apoio à pecuária  \
0                                         3635.0   
1                                         4475.0   
2                                         1095.0   
3                                          356.0   
4                                            0.0   
..                                           ...   
62                                           0.0   
63                                           0.0   
64                                           0.0   
65                                          29.0   
66                                           0.0   

    0280\nProdução florestal; pesca e aquicultura  \
0                                            78.0   
1                                            45.0   
2                                          1728.0   
3                                            11.0   
4                                             0.0   
..                                            ...   
62                                            0.0   
63                                            0.0   
64                                            0.0   
65                                           19.0   
66                                            0.0   

    0580\nExtração de carvão mineral e de minerais não metálicos  \
0                                                 0.0              
1                                                 0.0              
2                                                 2.0              
3                                               324.0 

Agora corrigimos os nomes das colunas

In [5]:
colunas = list(matriz_insumo_produto.columns)
for i in range(len(colunas)):
    colunas[i] = re.sub('\d', "", colunas[i])  # Se há algum número no nome da coluna, o substitua por nada ("")
    colunas[i] = re.sub('\n', " ", colunas[i])  # Se há algum "\n" (pular para nova linha) o substituia por um espaço (" ")
    if colunas[i][0] == " ":
        colunas[i] = colunas[i][1::]  # Se o primeiro caracter por um espaço, retire-o
    if colunas[i][-1] == " ":
        colunas[i] = colunas[i][:-1]  # Mesmo raciocínio, mas para o último caracter
        
matriz_insumo_produto.columns = colunas

VBP: Valor Bruto da Produção

DF: Demanda Final

In [6]:
# em caixa alta pois são valores constantes e não variáveis

VBP = matriz_insumo_produto["Demanda total"]
DF = matriz_insumo_produto["Demanda final"]

Retiramos agora as três primeiras colunas

``inplace=True`` aplica o método .drop() na própria matriz

In [7]:
matriz_insumo_produto.drop(columns=colunas[:3], axis=1, inplace=True)
matriz_insumo_produto.drop(columns=colunas[70:], inplace=True)

In [8]:
# ao colocar todos os índices iguais evitamos erros de processamento

matriz_insumo_produto.index = matriz_insumo_produto.columns
VBP.index = matriz_insumo_produto.columns
DF.index = matriz_insumo_produto.columns

Resolvido! Agora temos nossa matriz de insumo-produto pronta para ser manipulada!

In [9]:
import pymrio

Agora, importamos um pacote específico para análises de matriz de Insumo-Produto: o *pymrio*.

*Pymrio: Multi-Regional Input-Output Analysis in Python - A python module for automating io calculations and generating reports*

In [10]:
matriz_A = pymrio.calc_A(matriz_insumo_produto, VBP)
matriz_A

Agricultura, inclusive o apoio à agricultura e a pós-colheita  \
Agricultura, inclusive o apoio à agricultura e ...                                           0.022020               
Pecuária, inclusive o apoio à pecuária                                                       0.001723               
Produção florestal; pesca e aquicultura                                                      0.003027               
Extração de carvão mineral e de minerais não me...                                           0.000152               
Extração de petróleo e gás, inclusive as ativid...                                           0.000000               
...                                                                                               ...               
Saúde pública                                                                                0.000000               
Saúde privada                                                                                0.000000               
Atividades artísticas, criativas e de espetáculos                                            0.000000               
Organizações associativas e outros serviços pes...                                           0.000344               
Serviços domésticos                                                                          0.000000               

                                                    Pecuária, inclusive o apoio à pecuária  \
Agricultura, inclusive o apoio à agricultura e ...                                0.028196   
Pecuária, inclusive o apoio à pecuária                                            0.034711   
Produção florestal; pesca e aquicultura                                           0.008494   
Extração de carvão mineral e de minerais não me...                                0.002761   
Extração de petróleo e gás, inclusive as ativid...                                0.000000   
...                                                                                    ...   
Saúde pública                                                                     0.000000   
Saúde privada                                                                     0.000000   
Atividades artísticas, criativas e de espetáculos                                 0.000000   
Organizações associativas e outros serviços pes...                                0.000225   
Serviços domésticos                                                               0.000000   

                                                    Produção florestal; pesca e aquicultura  \
Agricultura, inclusive o apoio à agricultura e ...                                 0.002287   
Pecuária, inclusive o apoio à pecuária                                             0.001320   
Produção florestal; pesca e aquicultura                                            0.050673   
Extração de carvão mineral e de minerais não me...                                 0.000323   
Extração de petróleo e gás, inclusive as ativid...                                 0.000000   
...                                                                                     ...   
Saúde pública                                                                      0.000000   
Saúde privada                                                                      0.000000   
Atividades artísticas, criativas e de espetáculos                                  0.000000   
Organizações associativas e outros serviços pes...                                 0.000557   
Serviços domésticos                                                                0.000000   

                                                    Extração de carvão mineral e de minerais não metálicos  \
Agricultura, inclusive o apoio à agricultura e ...                                           0.000000        
Pecuária, inclusive o apoio à pecuária                                                       0.000000        
Produção florestal; pesca e aquicultura        

In [11]:
matriz_L = pymrio.calc_L(matriz_A)
matriz_L

Agricultura, inclusive o apoio à agricultura e a pós-colheita  \
Agricultura, inclusive o apoio à agricultura e ...                                           1.027523               
Pecuária, inclusive o apoio à pecuária                                                       0.002005               
Produção florestal; pesca e aquicultura                                                      0.004008               
Extração de carvão mineral e de minerais não me...                                           0.002909               
Extração de petróleo e gás, inclusive as ativid...                                           0.023635               
...                                                                                               ...               
Saúde pública                                                                                0.000000               
Saúde privada                                                                                0.000000               
Atividades artísticas, criativas e de espetáculos                                            0.000145               
Organizações associativas e outros serviços pes...                                           0.001689               
Serviços domésticos                                                                          0.000000               

                                                    Pecuária, inclusive o apoio à pecuária  \
Agricultura, inclusive o apoio à agricultura e ...                                0.056085   
Pecuária, inclusive o apoio à pecuária                                            1.041045   
Produção florestal; pesca e aquicultura                                           0.010536   
Extração de carvão mineral e de minerais não me...                                0.004904   
Extração de petróleo e gás, inclusive as ativid...                                0.020266   
...                                                                                    ...   
Saúde pública                                                                     0.000000   
Saúde privada                                                                     0.000000   
Atividades artísticas, criativas e de espetáculos                                 0.000270   
Organizações associativas e outros serviços pes...                                0.001780   
Serviços domésticos                                                               0.000000   

                                                    Produção florestal; pesca e aquicultura  \
Agricultura, inclusive o apoio à agricultura e ...                                 0.010155   
Pecuária, inclusive o apoio à pecuária                                             0.001761   
Produção florestal; pesca e aquicultura                                            1.053728   
Extração de carvão mineral e de minerais não me...                                 0.000822   
Extração de petróleo e gás, inclusive as ativid...                                 0.010180   
...                                                                                     ...   
Saúde pública                                                                      0.000000   
Saúde privada                                                                      0.000000   
Atividades artísticas, criativas e de espetáculos                                  0.000138   
Organizações associativas e outros serviços pes...                                 0.001250   
Serviços domésticos                                                                0.000000   

                                                    Extração de carvão mineral e de minerais não metálicos  \
Agricultura, inclusive o apoio à agricultura e ...                                           0.004839        
Pecuária, inclusive o apoio à pecuária                                                       0.000162        
Produção florestal; pesca e aquicultura        

Chegamos enfim à matriz de coeficientes!

Agora podemos exportar estas matrizes para arquivos csv, para evitar calcularmos novamente estes valores

In [12]:
import os

pasta = "Matrizes"

if not os.path.exists(pasta):
    # garante que a pasta onde quero salvar estes arquivos existe
    os.mkdir(pasta)

matriz_insumo_produto.to_csv("Matrizes\matriz_insumo_produto.csv")
matriz_A.to_csv("Matrizes\matriz_A.csv")
matriz_L.to_excel("Matrizes\matriz_coeficientes.xlsx")
VBP.to_csv("Matrizes\VBP.csv", header=True)
DF.to_csv("Matrizes\demanda_final.csv", header=True)

## Análise dos dados

---

Com a matriz de coeficientes pronta, podemos realizar as análises e descobrir o tipo dos setores

Os tipos dependem do coeficiente das ligações para frente e para trás. Calculamos isso da seguinte forma:

$$L_f = \frac{\frac{1}{n}\sum_{j}{b_{ij}}}{\frac{1}{n^2}\sum_i\sum_j b_{ij}}$$


$$L_t = \frac{\frac{1}{n}\sum_{i}{b_{ij}}}{\frac{1}{n^2}\sum_i\sum_j b_{ij}}$$

Ou seja: o coeficiente de ligação para frente da linha $i$ é igual à média dos coeficientes desta linha dividida pela média de todos os coeficientes da matriz;

E o coeficiente de ligações para trás da coluna $j$ é igual à média dos coeficientes desta coluna dividida pela média total dos coeficientes.

In [13]:
# é preciso usar .to_numpy().mean() para tirarmos a média de todos os valores do DataFrame de forma eficiente
media = matriz_L.to_numpy().mean()

l_f = matriz_L.mean(axis=0) / media # axis=0 para média das colunas
l_t = matriz_L.mean(axis=1) / media # axis=1 para média das colunas

In [14]:
df_coeficientes = pd.DataFrame({
    "Setor": list(matriz_L.columns),
    "Ligações para frente": l_f.to_numpy(),
    "Ligações para trás": l_t.to_numpy()
})
df_coeficientes

Setor  Ligações para frente  \
0   Agricultura, inclusive o apoio à agricultura e...              0.965247   
1              Pecuária, inclusive o apoio à pecuária              1.018823   
2             Produção florestal; pesca e aquicultura              0.742157   
3   Extração de carvão mineral e de minerais não m...              0.995418   
4   Extração de petróleo e gás, inclusive as ativi...              0.963418   
..                                                ...                   ...   
62                                      Saúde pública              0.814404   
63                                      Saúde privada              0.840359   
64  Atividades artísticas, criativas e de espetáculos              0.873071   
65  Organizações associativas e outros serviços pe...              0.902033   
66                                Serviços domésticos              0.553941   

    Ligações para trás  
0             1.862446  
1             0.870680  
2             0.762387  
3             0.688005  
4             1.406660  
..                 ...  
62            0.553941  
63            0.611399  
64            0.640074  
65            0.724536  
66            0.553941  

[67 rows x 3 columns]

Podemos dividir os setores em quatro tipos:

- **Tipo I:** ($L_t < 1; L_f < 1$) Geralmente independente, sem ligações ou ligações fracas;

- **Tipo II:** ($L_t < 1; L_f > 1$) Depende da demanda interisdustrial, mais relações à frente;

- **Tipo III:** ($L_t > 1; L_f > 1$) Geralmente dependente, é um ***setor chave***;

- **Tipo IV:** ($L_t > 1; L_f < 1$) Depende da oferta interindustrial, mais relações para trás;

In [15]:
tipos = []
chaves = []

for i in range(df_coeficientes.shape[0]):
    chave = ''
    if df_coeficientes.iloc[i, 1] < 1 and df_coeficientes.iloc[i, 2] < 1:
        tipo = 'I'
    elif df_coeficientes.iloc[i, 1] < 1 and df_coeficientes.iloc[i, 2] > 1:
        tipo = 'II'
    elif df_coeficientes.iloc[i, 1] > 1 and df_coeficientes.iloc[i, 2] > 1:
        tipo = 'III'
        chave = '*'
    else:
        tipo = 'IV'
    tipos.append(tipo)
    chaves.append(chave)

df_coeficientes['Tipo'] = tipos
df_coeficientes['Setor Chave'] = chaves
df_coeficientes[df_coeficientes['Setor Chave'] != '']

Setor  Ligações para frente  \
16  Fabricação de celulose, papel e produtos de papel              1.132302   
18                     Refino de petróleo e coquerias              1.350624   
20  Fabricação de químicos orgânicos e inorgânicos...              1.115801   
21  Fabricação de defensivos, desinfestantes, tint...              1.121799   
24  Fabricação de produtos de borracha e de materi...              1.143552   
26  Produção de ferro gusa/ferroligas, siderurgia ...              1.222849   
28  Fabricação de produtos de metal, exceto máquin...              1.077724   
37  Energia elétrica, gás natural e outras utilidades              1.162158   
41                               Transporte terrestre              1.121909   

    Ligações para trás Tipo Setor Chave  
16            1.008885  III           *  
18            2.823324  III           *  
20            1.909688  III           *  
21            1.134049  III           *  
24            1.202437  III           *  
26            1.043301  III           *  
28            1.053088  III           *  
37            2.027143  III           *  
41            2.251273  III           *

É notável um padrão entre os possíveis setores chave: *a maioria deles é do setor industrial*. Um ponto muito comentado por economistas desenvolvimentistas é a importância da retomada do crescimento deste setor no Brasil para alavancar a economia.

Uma relação interessante que podemos visualizar é a seguinte:

$$L \cdot DF = VBP$$

Isto é, a multiplicação da matriz de coeficientes pelo vetor da Demanda Final resulta no VBP! Com isso, podemos prever qual seria a variação do VBP ao mudarmos cada valor no DF.

Vejamos, qual seria o impactor no VBP se a Demanda Final apenas do primeiro setor subisse em 10% e dos outros permanecesse constante, considerando a matriz de coeficientes que nós temos?

In [16]:
# .copy impede erros de memória ao copiar um DataFrame ou Series
multiplicador = [1.1]
multiplicador.extend([1]*66)
np.array(multiplicador) 

DF_2 = DF.copy() * multiplicador 
DF_2.index = matriz_L.columns
VBP_2 = matriz_L.dot(DF_2)
VBP_2 / VBP

Agricultura, inclusive o apoio à agricultura e a pós-colheita    1.056305
Pecuária, inclusive o apoio à pecuária                           1.000258
Produção florestal; pesca e aquicultura                          1.001947
Extração de carvão mineral e de minerais não metálicos           1.002366
Extração de petróleo e gás, inclusive as atividades de apoio     1.002415
                                                                   ...   
Saúde pública                                                    1.000000
Saúde privada                                                    1.000000
Atividades artísticas, criativas e de espetáculos                1.000071
Organizações associativas e outros serviços pessoais             1.000184
Serviços domésticos                                              1.000000
Length: 67, dtype: float64

Podemos ver, como esperado, que a variação é diferente em cada setor e não batem com a variação na Demanda.